In [ ]:
import torch
print(f"CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Dispositivo actual: {torch.cuda.get_device_name(0)}")

In [ ]:
# notebooks/experiment1.ipynb

import os
import sys
from pathlib import Path

# Agrega el directorio src al path de Python
project_dir = Path.cwd().parent
sys.path.append(str(project_dir))

import torch
import pytorch_lightning as L
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
import wandb
from omegaconf import OmegaConf
import hydra
from src.models import (
    UNetAutoencoder, ButterflyClassifier, LitAutoencoder,
    LitClassifier, create_encoder_classifier
)
from src.data import ButterflyDataModule
from src.utils import (
    set_seed, evaluate_model_performance, compare_model_versions,
    save_experiment_results, log_batch_predictions
)

# Configuración inicial
DATASET_PATH = Path('Butterfly-dataset')
CONFIG_PATH = Path('conf/config.yaml')

# Carga la configuración de Hydra
config = OmegaConf.load(CONFIG_PATH)

# Configuración de dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Establece semilla para reproducibilidad
set_seed(config.seed)

def train_experiment(unlabeled_ratio: float = 0.7) -> dict:
    """Ejecuta el experimento completo."""
    
    # Inicializa wandb
    experiment_name = f"exp1_{int(unlabeled_ratio*100)}"
    wandb.init(
        project=config.wandb.project,
        name=experiment_name,
        config=OmegaConf.to_container(config, resolve=True)
    )
    
    # Inicializa el módulo de datos
    data_module = ButterflyDataModule(
        data_dir=str(DATASET_PATH),
        batch_size=config.data.batch_size,
        unlabeled_ratio=unlabeled_ratio,
        image_size=config.data.image_size,
        num_workers=config.data.num_workers
    )
    data_module.setup()
    
    # Entrenamiento del autoencoder
    print("\nEntrenando Autoencoder...")
    autoencoder = UNetAutoencoder(
        in_channels=3,
        latent_dim=config.model.autoencoder.latent_dim
    )
    lit_autoencoder = LitAutoencoder(
        autoencoder=autoencoder,
        learning_rate=config.training.learning_rate
    )
    
    # Callbacks para el autoencoder
    ae_callbacks = [
        EarlyStopping(
            monitor='train_loss',
            patience=config.training.early_stopping_patience,
            mode='min'
        ),
        ModelCheckpoint(
            dirpath='checkpoints',
            filename=f'{experiment_name}_autoencoder_best',
            monitor='train_loss',
            mode='min'
        )
    ]
    
    # Entrenador para el autoencoder
    trainer_ae = L.Trainer(
        max_epochs=config.training.max_epochs,
        callbacks=ae_callbacks,
        accelerator='gpu' if torch.cuda.is_available() else 'cpu',
        devices=1
    )
    
    # Entrenamiento del autoencoder
    trainer_ae.fit(lit_autoencoder, data_module.unlabeled_dataloader())
    
    # Entrenamiento de clasificadores
    results = {}
    
    # Clasificador A (desde cero)
    print("\nEntrenando Clasificador A (desde cero)...")
    classifier_a = ButterflyClassifier(num_classes=data_module.num_classes)
    lit_classifier_a = LitClassifier(
        classifier=classifier_a,
        learning_rate=config.training.learning_rate,
        weight_decay=config.training.weight_decay,
        class_weights=data_module.class_weights
    )
    
    trainer_a = L.Trainer(
        max_epochs=config.training.max_epochs,
        callbacks=[
            EarlyStopping(monitor='val_loss', patience=config.training.early_stopping_patience),
            ModelCheckpoint(
                dirpath='checkpoints',
                filename=f'{experiment_name}_classifier_a_best',
                monitor='val_loss'
            )
        ],
        accelerator='gpu' if torch.cuda.is_available() else 'cpu',
        devices=1
    )
    
    trainer_a.fit(lit_classifier_a, data_module)
    results['classifier_a'] = trainer_a.test(lit_classifier_a, datamodule=data_module)[0]
    
    # Clasificador B1 (encoder congelado)
    print("\nEntrenando Clasificador B1 (encoder congelado)...")
    classifier_b1 = create_encoder_classifier(
        autoencoder=autoencoder,
        num_classes=data_module.num_classes,
        freeze_encoder=True
    )
    lit_classifier_b1 = LitClassifier(
        classifier=classifier_b1,
        learning_rate=config.training.learning_rate,
        weight_decay=config.training.weight_decay,
        class_weights=data_module.class_weights
    )
    
    trainer_b1 = L.Trainer(
        max_epochs=config.training.max_epochs,
        callbacks=[
            EarlyStopping(monitor='val_loss', patience=config.training.early_stopping_patience),
            ModelCheckpoint(
                dirpath='checkpoints',
                filename=f'{experiment_name}_classifier_b1_best',
                monitor='val_loss'
            )
        ],
        accelerator='gpu' if torch.cuda.is_available() else 'cpu',
        devices=1
    )
    
    trainer_b1.fit(lit_classifier_b1, data_module)
    results['classifier_b1'] = trainer_b1.test(lit_classifier_b1, datamodule=data_module)[0]
    
    # Clasificador B2 (encoder entrenable)
    print("\nEntrenando Clasificador B2 (encoder entrenable)...")
    classifier_b2 = create_encoder_classifier(
        autoencoder=autoencoder,
        num_classes=data_module.num_classes,
        freeze_encoder=False
    )
    lit_classifier_b2 = LitClassifier(
        classifier=classifier_b2,
        learning_rate=config.training.learning_rate,
        weight_decay=config.training.weight_decay,
        class_weights=data_module.class_weights
    )
    
    trainer_b2 = L.Trainer(
        max_epochs=config.training.max_epochs,
        callbacks=[
            EarlyStopping(monitor='val_loss', patience=config.training.early_stopping_patience),
            ModelCheckpoint(
                dirpath='checkpoints',
                filename=f'{experiment_name}_classifier_b2_best',
                monitor='val_loss'
            )
        ],
        accelerator='gpu' if torch.cuda.is_available() else 'cpu',
        devices=1
    )
    
    trainer_b2.fit(lit_classifier_b2, data_module)
    results['classifier_b2'] = trainer_b2.test(lit_classifier_b2, datamodule=data_module)[0]
    
    # Cuantización y comparación de modelos
    classifiers = {
        'classifier_a': lit_classifier_a.classifier,
        'classifier_b1': lit_classifier_b1.classifier,
        'classifier_b2': lit_classifier_b2.classifier
    }
    
    print("\nRealizando cuantización y comparación de modelos...")
    quantization_results = {}
    
    for name, model in classifiers.items():
        print(f"\nProcesando {name}...")
        # Cuantizar modelo
        quantized_model = torch.quantization.quantize_dynamic(
            model,
            {torch.nn.Linear, torch.nn.Conv2d},
            dtype=torch.qint8
        )
        
        # Comparar versiones
        comparison = compare_model_versions(
            original_model=model,
            quantized_model=quantized_model,
            test_dataloader=data_module.test_dataloader(),
            device=device
        )
        
        quantization_results[name] = comparison
        
        # Log a wandb
        wandb.log({
            f"{name}/original_accuracy": comparison['original_metrics']['accuracy'],
            f"{name}/quantized_accuracy": comparison['quantized_metrics']['accuracy'],
            f"{name}/size_reduction": comparison['size_reduction_ratio'],
            f"{name}/latency_improvement": comparison['latency_improvement']
        })
    
    # Guardar resultados
    final_results = {
        'training_results': results,
        'quantization_results': quantization_results,
        'config': OmegaConf.to_container(config, resolve=True)
    }
    
    save_experiment_results(
        results=final_results,
        save_dir=Path('results'),
        experiment_name=experiment_name
    )
    
    wandb.finish()
    return final_results

if __name__ == "__main__":
    # Ejecutar experimento con diferentes ratios
    ratios = [0.7, 0.9]
    all_results = {}
    
    for ratio in ratios:
        print(f"\nEjecutando experimento con {ratio*100}% datos sin etiquetas")
        results = train_experiment(ratio)
        all_results[f"ratio_{ratio}"] = results
        
        # Imprimir resultados
        print(f"\nResultados para ratio {ratio}:")
        for model_name, metrics in results['training_results'].items():
            print(f"\n{model_name}:")
            print(f"Test accuracy: {metrics['test_acc']:.4f}")
            print(f"Test loss: {metrics['test_loss']:.4f}")
            
            quant_results = results['quantization_results'][model_name]
            print(f"Size reduction: {quant_results['size_reduction_ratio']*100:.2f}%")
            print(f"Latency improvement: {quant_results['latency_improvement']*100:.2f}%")
            print(f"Accuracy impact: {quant_results['accuracy_difference']*100:.2f}%")

    # Guardar resultados globales
    import json
    with open('results/all_results.json', 'w') as f:
        json.dump(all_results, f, indent=4)